In [ ]:
import os
import glob
import PyPDF2
import pdfplumber
from pathlib import Path
import re
import json
from datetime import datetime
import sys

class PDFToTextExtractor:
    def __init__(self, pdf_folder_path):
        """
        Initialize the PDF to text extractor
        
        Args:
            pdf_folder_path: Path to folder containing PDF files
        """
        self.pdf_folder = pdf_folder_path
        
        # Create output folder with same name + "_txt"
        folder_name = os.path.basename(self.pdf_folder.rstrip(os.sep))
        parent_dir = os.path.dirname(self.pdf_folder)
        self.txt_folder = os.path.join(parent_dir, f"{folder_name}_txt")
        
        # Setup folders
        self.setup_folders()
        
        # Statistics
        self.stats = {
            'total_pdfs': 0,
            'successful': 0,
            'failed': 0,
            'total_words': 0,
            'total_pages': 0,
            'failed_files': []
        }
    
    def setup_folders(self):
        """Create necessary folders"""
        # Ensure PDF folder exists
        if not os.path.exists(self.pdf_folder):
            print(f"❌ Error: PDF folder not found: {self.pdf_folder}")
            print(f"Current directory: {os.getcwd()}")
            sys.exit(1)
        
        # Create text folder if it doesn't exist
        if not os.path.exists(self.txt_folder):
            os.makedirs(self.txt_folder)
            print(f"📁 Created text folder: {self.txt_folder}")
    
    def get_all_pdfs(self):
        """Get all PDF files from the folder"""
        # Get all PDF files (case insensitive)
        pdf_patterns = [
            os.path.join(self.pdf_folder, "*.pdf"),
            os.path.join(self.pdf_folder, "*.PDF"),
        ]
        
        pdf_files = []
        for pattern in pdf_patterns:
            pdf_files.extend(glob.glob(pattern))
        
        # Remove duplicates and sort
        pdf_files = list(set(pdf_files))
        pdf_files.sort()
        
        self.stats['total_pdfs'] = len(pdf_files)
        return pdf_files
    
    def analyze_pdf_type(self, pdf_path):
        """Analyze PDF to determine best extraction method"""
        try:
            # Try PyPDF2 first
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                # Check if PDF is encrypted
                if pdf_reader.is_encrypted:
                    return "encrypted", len(pdf_reader.pages), 0
                
                # Test first few pages for text
                text_sample = ""
                pages_to_test = min(3, len(pdf_reader.pages))
                
                for i in range(pages_to_test):
                    page_text = pdf_reader.pages[i].extract_text()
                    if page_text:
                        text_sample += page_text
                
                # Check if we got substantial text
                if len(text_sample.strip()) > 200:
                    return "PyPDF2", len(pdf_reader.pages), len(text_sample.split())
                else:
                    return "pdfplumber", len(pdf_reader.pages), len(text_sample.split())
                    
        except Exception as e:
            print(f"   ⚠️ Analysis error: {str(e)[:80]}")
            return "pdfplumber", 0, 0
    
    def extract_with_pypdf2(self, pdf_path):
        """Extract text using PyPDF2"""
        text = ""
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                
                # Try to decrypt if encrypted
                if pdf_reader.is_encrypted:
                    try:
                        pdf_reader.decrypt("")  # Try empty password
                    except:
                        pass
                
                for page_num, page in enumerate(pdf_reader.pages, 1):
                    page_text = page.extract_text()
                    if page_text:
                        # Clean the text
                        page_text = self.clean_text(page_text)
                        text += f"\n{'='*60}\nPAGE {page_num}\n{'='*60}\n\n{page_text}\n"
                
            return text, len(pdf_reader.pages)
            
        except Exception as e:
            raise Exception(f"PyPDF2 extraction failed: {e}")
    
    def extract_with_pdfplumber(self, pdf_path):
        """Extract text using pdfplumber"""
        text = ""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page_num, page in enumerate(pdf.pages, 1):
                    # Try direct text extraction first
                    page_text = page.extract_text()
                    
                    # If no text or very little, try word extraction
                    if not page_text or len(page_text.strip()) < 50:
                        words = page.extract_words()
                        if words:
                            # Sort words by reading order (top to bottom, left to right)
                            words_sorted = sorted(words, key=lambda x: (x['top'], x['x0']))
                            page_text = ' '.join([w['text'] for w in words_sorted])
                    
                    if page_text:
                        page_text = self.clean_text(page_text)
                        text += f"\n{'='*60}\nPAGE {page_num}\n{'='*60}\n\n{page_text}\n"
                
            return text, len(pdf.pages)
            
        except Exception as e:
            raise Exception(f"pdfplumber extraction failed: {e}")
    
    def clean_text(self, text):
        """Clean and normalize extracted text"""
        if not text:
            return ""
        
        # Remove null characters
        text = text.replace('\x00', '')
        
        # Replace multiple spaces/newlines with single space
        text = re.sub(r'\s+', ' ', text)
        
        # Fix common PDF/OCR issues
        replacements = {
            'ﬁ': 'fi',
            'ﬂ': 'fl',
            'ﬀ': 'ff',
            'ﬃ': 'ffi',
            'ﬄ': 'ffl',
            '�': '',  # Remove replacement characters
            '\uf0b7': '•',  # Bullet points
        }
        
        for old, new in replacements.items():
            text = text.replace(old, new)
        
        # Remove extra spaces around punctuation
        text = re.sub(r'\s+([.,;:!?])', r'\1', text)
        text = re.sub(r'([.,;:!?])\s+', r'\1 ', text)
        
        # Ensure proper paragraph separation
        text = re.sub(r'\.\s+([A-Z])', r'. \n\n\1', text)
        
        return text.strip()
    
    def add_metadata_header(self, text, pdf_name, pages, method, word_count):
        """Add metadata header to the text file"""
        header = f"""{'='*70}
PDF TO TEXT CONVERSION REPORT
{'='*70}

SOURCE PDF: {pdf_name}
CONVERSION DATE: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
EXTRACTION METHOD: {method}
TOTAL PAGES: {pages}
ESTIMATED WORD COUNT: {word_count:,}
{'='*70}

TEXT CONTENT:
{'='*70}

"""
        return header + text
    
    def convert_single_pdf(self, pdf_path):
        """Convert a single PDF file to text"""
        pdf_name = os.path.basename(pdf_path)
        txt_name = os.path.splitext(pdf_name)[0] + ".txt"
        txt_path = os.path.join(self.txt_folder, txt_name)
        
        print(f"📄 Processing: {pdf_name}")
        
        try:
            # Analyze PDF to choose best method
            method, total_pages, word_sample = self.analyze_pdf_type(pdf_path)
            
            if method == "encrypted":
                raise Exception("PDF is encrypted and cannot be read")
            
            # Extract text using chosen method
            if method == "PyPDF2":
                text, pages_extracted = self.extract_with_pypdf2(pdf_path)
            else:  # pdfplumber
                text, pages_extracted = self.extract_with_pdfplumber(pdf_path)
            
            # Update page count if analysis failed
            if total_pages == 0 and pages_extracted > 0:
                total_pages = pages_extracted
            
            # Check if we got any text
            if not text.strip():
                raise Exception("No text could be extracted from the PDF")
            
            # Calculate statistics
            word_count = len(text.split())
            char_count = len(text)
            
            # Add metadata header
            final_text = self.add_metadata_header(
                text, pdf_name, total_pages, method, word_count
            )
            
            # Save to text file
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(final_text)
            
            # Update statistics
            self.stats['successful'] += 1
            self.stats['total_pages'] += total_pages
            self.stats['total_words'] += word_count
            
            print(f"   ✅ Saved: {txt_name}")
            print(f"   📊 Pages: {total_pages}, Words: {word_count:,}, Method: {method}")
            
            return True, txt_path
            
        except Exception as e:
            error_msg = str(e)
            print(f"   ❌ Failed: {error_msg[:80]}")
            
            # Save error information
            self.stats['failed'] += 1
            self.stats['failed_files'].append({
                'pdf_name': pdf_name,
                'error': error_msg
            })
            
            # Create error log file
            error_log = os.path.join(self.txt_folder, f"ERROR_{txt_name}")
            with open(error_log, 'w', encoding='utf-8') as f:
                f.write(f"Failed to convert: {pdf_name}\n")
                f.write(f"Error: {error_msg}\n")
                f.write(f"Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            
            return False, error_msg
    
    def convert_all_pdfs(self):
        """Convert all PDFs in the folder"""
        print("=" * 80)
        print("📄 PDF TO TEXT CONVERSION TOOL")
        print("=" * 80)
        
        # Get all PDF files
        pdf_files = self.get_all_pdfs()
        
        if not pdf_files:
            print(f"\n❌ No PDF files found in: {self.pdf_folder}")
            print("   Please check the folder path and ensure it contains PDF files.")
            return False
        
        print(f"\n📁 PDF Folder: {self.pdf_folder}")
        print(f"📁 Text Folder: {self.txt_folder}")
        print(f"📚 Found {len(pdf_files)} PDF file(s)")
        print("=" * 80)
        
        # Process each PDF
        for idx, pdf_path in enumerate(pdf_files, 1):
            print(f"\n[{idx}/{len(pdf_files)}] ", end="")
            self.convert_single_pdf(pdf_path)
            
            # Show progress
            if idx % 10 == 0 or idx == len(pdf_files):
                print(f"\n📈 Progress: {idx}/{len(pdf_files)} processed "
                      f"({self.stats['successful']} successful, {self.stats['failed']} failed)")
        
        # Print final summary
        self.print_summary()
        
        # Save detailed report
        self.save_detailed_report()
        
        return True
    
    def print_summary(self):
        """Print conversion summary"""
        print("\n" + "=" * 80)
        print("📊 CONVERSION SUMMARY")
        print("=" * 80)
        
        success_rate = (self.stats['successful'] / self.stats['total_pdfs'] * 100) if self.stats['total_pdfs'] > 0 else 0
        
        print(f"📁 PDF Files Found: {self.stats['total_pdfs']:,}")
        print(f"✅ Successfully Converted: {self.stats['successful']:,}")
        print(f"❌ Failed: {self.stats['failed']:,}")
        print(f"📈 Success Rate: {success_rate:.1f}%")
        print(f"📄 Total Pages Processed: {self.stats['total_pages']:,}")
        print(f"📝 Total Words Extracted: {self.stats['total_words']:,}")
        
        if self.stats['successful'] > 0:
            avg_words = self.stats['total_words'] // self.stats['successful']
            avg_pages = self.stats['total_pages'] // self.stats['successful']
            print(f"📊 Average per Document: {avg_words:,} words, {avg_pages} pages")
        
        print(f"\n📂 Text Files Location: {os.path.abspath(self.txt_folder)}")
        
        # Show failed files if any
        if self.stats['failed_files']:
            print(f"\n⚠️  Failed Files ({len(self.stats['failed_files'])}):")
            for fail in self.stats['failed_files'][:5]:  # Show first 5
                print(f"   • {fail['pdf_name']}: {fail['error'][:60]}...")
            if len(self.stats['failed_files']) > 5:
                print(f"   ... and {len(self.stats['failed_files']) - 5} more")
    
    def save_detailed_report(self):
        """Save detailed conversion report as JSON"""
        report = {
            'conversion_report': {
                'timestamp': datetime.now().isoformat(),
                'pdf_folder': self.pdf_folder,
                'txt_folder': self.txt_folder,
                'statistics': self.stats,
                'system_info': {
                    'python_version': sys.version,
                    'platform': sys.platform,
                    'current_directory': os.getcwd()
                }
            }
        }
        
        report_file = os.path.join(self.txt_folder, "conversion_report.json")
        with open(report_file, 'w', encoding='utf-8') as f:
            json.dump(report, f, indent=2, ensure_ascii=False)
        
        print(f"\n📋 Detailed report saved to: {report_file}")

def install_requirements():
    """Install required packages if not already installed"""
    required_packages = ['PyPDF2', 'pdfplumber']
    
    for package in required_packages:
        try:
            __import__(package)
            print(f"✅ {package} is already installed")
        except ImportError:
            print(f"📦 Installing {package}...")
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installed successfully")

def main():
    """Main function"""
    print("=" * 80)
    print("🏛️  SUPREME COURT JUDGEMENTS - PDF TO TEXT CONVERTER")
    print("=" * 80)
    
    # Install required packages
    print("\n🔍 Checking dependencies...")
    install_requirements()
    
    # Set PDF folder path
    pdf_folder = r"C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements"
    
    # Verify the folder exists
    if not os.path.exists(pdf_folder):
        print(f"\n❌ Error: Folder not found: {pdf_folder}")
        print("Please verify the path exists and contains PDF files.")
        
        # Suggest alternative
        current_dir = os.getcwd()
        print(f"\nCurrent working directory: {current_dir}")
        print("You might need to use one of these paths:")
        print(f"1. {os.path.join(current_dir, 'supreme_court_judgements')}")
        print(f"2. {os.path.join(os.path.expanduser('~'), 'Desktop', 'AI Project', 'supreme_court_judgements')}")
        
        # Ask for alternative path
        alt_path = input("\nEnter correct folder path (or press Enter to exit): ").strip()
        if alt_path:
            pdf_folder = alt_path
        else:
            return
    
    # Create and run extractor
    print(f"\n🚀 Starting conversion process...")
    extractor = PDFToTextExtractor(pdf_folder)
    
    # Start conversion
    success = extractor.convert_all_pdfs()
    
    if success:
        print("\n" + "=" * 80)
        print("🎉 CONVERSION COMPLETED SUCCESSFULLY!")
        print("=" * 80)
        
        # Show folder contents
        txt_folder = extractor.txt_folder
        if os.path.exists(txt_folder):
            txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]
            error_files = [f for f in os.listdir(txt_folder) if f.startswith('ERROR_')]
            
            print(f"\n📂 Created {len(txt_files)} text files in:")
            print(f"   {os.path.abspath(txt_folder)}")
            
            if txt_files:
                print(f"\n📋 First 10 text files created:")
                for file in txt_files[:10]:
                    file_path = os.path.join(txt_folder, file)
                    size_kb = os.path.getsize(file_path) // 1024
                    print(f"   • {file} ({size_kb} KB)")
                
                if len(txt_files) > 10:
                    print(f"   ... and {len(txt_files) - 10} more")
            
            if error_files:
                print(f"\n⚠️  {len(error_files)} error logs created")
    else:
        print("\n❌ Conversion encountered errors. Check the summary above.")

def quick_convert():
    """Simple one-line function to run the conversion"""
    pdf_folder = r"C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements"
    
    # Verify folder exists
    if not os.path.exists(pdf_folder):
        print(f"Error: Folder not found: {pdf_folder}")
        return False
    
    print(f"Converting PDFs from: {pdf_folder}")
    extractor = PDFToTextExtractor(pdf_folder)
    return extractor.convert_all_pdfs()

if __name__ == "__main__":
    # You can run either:
    # 1. main() - for interactive mode with prompts
    # 2. quick_convert() - for direct execution
    
    # Run interactive mode
    main()
    
    # Or for quick execution, uncomment this:
    # quick_convert()

🏛️  SUPREME COURT JUDGEMENTS - PDF TO TEXT CONVERTER

🔍 Checking dependencies...
✅ PyPDF2 is already installed
✅ pdfplumber is already installed

🚀 Starting conversion process...
📁 Created text folder: C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements_txt
📄 PDF TO TEXT CONVERSION TOOL

📁 PDF Folder: C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements
📁 Text Folder: C:\Users\Armaghan Rafique\Desktop\AI Project\supreme_court_judgements_txt
📚 Found 1474 PDF file(s)

[1/1474] 📄 Processing: 1113.pdf
   ❌ Failed: No text could be extracted from the PDF

[2/1474] 📄 Processing: 1_Tajuddin_2_Gul_Alam_sons_of_Mir_Alam_VERSUS_M__5_._20tajuddin_20etc_20vs_20Mst_20Zainab.pdf
   ✅ Saved: 1_Tajuddin_2_Gul_Alam_sons_of_Mir_Alam_VERSUS_M__5_._20tajuddin_20etc_20vs_20Mst_20Zainab.txt
   📊 Pages: 9, Words: 1,755, Method: PyPDF2

[3/1474] 📄 Processing: 2025LHC7211.pdf
   ✅ Saved: 2025LHC7211.txt
   📊 Pages: 16, Words: 7,212, Method: PyPDF2

[4/1474] 📄 Processing: 2025

Multiple definitions in dictionary at byte 0x5ec876 for key /Info
Multiple definitions in dictionary at byte 0x5ec883 for key /Info
Multiple definitions in dictionary at byte 0x5ec890 for key /Info


   ✅ Saved: 5. final judgment of nafeesa kiran.txt
   📊 Pages: 6, Words: 1,468, Method: PyPDF2

[57/1474] 📄 Processing: 728-16.pdf
   ❌ Failed: No text could be extracted from the PDF

[58/1474] 📄 Processing: ADMINISTRATIVE_APPEALREPRESENTATION_NO012013_administrative_20appeal.pdf
   ✅ Saved: ADMINISTRATIVE_APPEALREPRESENTATION_NO012013_administrative_20appeal.txt
   📊 Pages: 7, Words: 2,397, Method: PyPDF2

[59/1474] 📄 Processing: Abbas_and_others_versus_Raza_others_Abbas_20__20others_20versus_20Raza_20__20others_20SKD_20Branch.pdf
   ✅ Saved: Abbas_and_others_versus_Raza_others_Abbas_20__20others_20versus_20Raza_20__20others_20SKD_20Branch.txt
   📊 Pages: 6, Words: 1,220, Method: PyPDF2

[60/1474] 📄 Processing: Abdu-Rahim_Shah_Versus_Provincial_Government_oth_Abdur-Rahim_20Shah_20versus_20the_20Prov_20Govt.pdf
   ✅ Saved: Abdu-Rahim_Shah_Versus_Provincial_Government_oth_Abdur-Rahim_20Shah_20versus_20the_20Prov_20Govt.txt
   📊 Pages: 4, Words: 931, Method: PyPDF2

📈 Progress: 60/1474 